In [ ]:
import pandas as pd


In [ ]:
mov = pd.read_csv("tmdb_5000_movies.csv")
cred = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
mov.head(2)

In [ ]:
mov.shape

In [ ]:
cred.head()

In [ ]:
cred.shape

In [ ]:
cred.rename(columns={'title': 'original_title'}, inplace=True)

In [ ]:
mov = mov.merge(cred, on="original_title")

In [ ]:
mov.head(2)

In [ ]:
mov = mov[['budget', 'homepage', 'id', 'original_language', 'original_title' , 'genres', 'popularity', 'production_companies', 'cast', 'crew']]

In [ ]:
mov.head()

In [ ]:
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
mov.dropna(inplace=True)

In [ ]:
mov['genres'] = mov['genres'].apply(convert) 
mov.head()

In [ ]:
def convert3(text):
    L = []
    count = 0
    for i in ast.literal_eval(text):
        if count < 3: 
             L.append(i['name'])
             count += 1
    return L

In [ ]:
mov['cast'] = mov['cast'].apply(convert3)
mov.head()

In [ ]:
def fetch_dir(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
mov['crew'] = mov['crew'].apply(fetch_dir) 

In [ ]:
mov.head()

In [ ]:
def fetch_prod(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

In [ ]:
mov['production_companies'] = mov['production_companies'].apply(fetch_prod) 

In [ ]:
mov.head()

In [ ]:
def remGap(L):
    L1 = []
    for i in L:
        L1. append(i.replace(" ", ""))

    return L1
    

In [ ]:
mov["cast"] = mov["cast"].apply(remGap)
mov["crew"] = mov["crew"].apply(remGap)
mov["genres"] = mov["genres"].apply(remGap)
mov["production_companies"] = mov["production_companies"].apply(remGap)

In [ ]:
mov.head()

In [ ]:
mov['tags'] = mov['genres'] + mov['cast'] + mov['crew'] + mov['production_companies']

In [ ]:
new = mov.drop(columns = ['genres', 'cast', 'crew'])

In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = "english")

In [ ]:
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
new[new['original_title'] == 'Batman Begins'].index[0]

In [ ]:
def recommend(mov):
    if mov not in new['original_title'].values:
        print(f"Movie '{mov}' not found in the dataset.")
        return

    index = new[new['original_title'] == mov].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    
    for i in distances[1:6]:
        print(new.iloc[i[0]].original_title)


In [ ]:
recommend('Shrek')


In [ ]:
new.iloc[1206].original_title

In [ ]:
import pickle

In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))